In [14]:
!pip install PyMuPDF openai
!pip install --upgrade openai

In [25]:
import fitz  # PyMuPDF
import openai

def extract_text_from_pdf(pdf_path):
    # Open the provided PDF file
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text



In [26]:
# Path to your PDF file
pdf_path = "/content/OREA-Form-100 (2)Agreement of Purchase and Sale.pdf"
# here to give path of contract--- example

# Extract text from the PDF
contract_text = extract_text_from_pdf(pdf_path)

In [18]:
from openai import OpenAI
API_KEY = ''
### Here to give API
client = OpenAI(
  api_key=API_KEY
 )

In [19]:
with open('./sample_contract.txt') as f:
  contract_text=f.read()


In [20]:
def get_template(content,questions,answers=''):
  template = """
  ### Contract Content:
  {}

  ### Questions:
  {}
  ### Answer to each question:
  {}

  """.format(content,'\n'.join(questions),'\n'.join(answers) if answers else '')
  return template

In [27]:
questions = [
    "What is the address of the property?",
    "What is the deadline for the closing?",
    "Is it possible for the buyer or seller to delay the closing by a few days?",
    "Can extrinsic evidence be considered admissible in relation to the contract?",
    "Is the buyer or the seller responsible for fees associated with title transfer?"
]


In [29]:
answers=['Property Address: 1 Main St, New York, New York','Closing Date: January 1, 2022','No. Because time is of the essence.', 'No.','Seller is responsible']
Example= get_template(contract_text,questions,answers)

In [23]:
responses = []

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "We will give you some contracts and Please answer the following inquiries: ”What is the address of the property?”, ”What is the deadline for the closing?”, ”Is it possible for the buyer or seller to delay the closing by a few days?”, etc. If the infomation is not provided, just imply by law. "},
    {"role":"system","content":f"Here is an example: {Example}"},
    {"role": "user", "content": get_template(contract_text,questions)}
  ],
  max_tokens=200
)

responses.append(completion.choices[0].message.content)


In [24]:
print(responses[0])

What is the address of the property?  
- Property Address: 1 Main St, New York, New York

What is the deadline for the closing?  
- Closing Date: January 1, 2022

Is it possible for the buyer or seller to delay the closing by a few days?  
- No. Because time is of the essence.

Can extrinsic evidence be considered admissible in relation to the contract?  
- No.

Is the buyer or the seller responsible for fees associated with title transfer?  
- Seller is responsible
